In [1]:
# train_test_split
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import catboost

In [2]:
# not using image data for now
# there are total of 1451 rows in dataset, I have removed 49 rows for which image was not downloadable
df = pd.read_csv('../data/processed/cleaned_results.csv')
X = df[['name','brandName']]
Y = df['categoryName']

# Cannot use Statrify here since there are categories with only 1 value, so it's not possible to split them
X_train,X_test, y_train,y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [3]:
X_train

,name,brandName
1287,Serge Lutens La Fille De Berlin Eau De Parfum ...,Serge Lutens
1035,Fanola Oro Puro Therapy Color Keratin 100ml - ...,Fanola
416,Virgin Island Water by Creed Eau De Parfum 50ml,Creed
1166,Redken Color Extend Magnetics Conditioner for ...,Redken
86,L'OCCITANE Cade Energizing Fluid 50ml Fresh Fa...,L'Occitane
...,...,...
1095,DIOR DIORSKIN FOREVER SKIN Concealer 3N,Dior
1130,Stabilizing Repair Cream,Dermalogica
1294,Wella Koleston Perfect Me + Pure Naturals 10/0...,Wella
860,L'Occitane Immortelle Divine Eye Balm 0.50 oz,L'Occitane


### no pre-processing

In [4]:
# using Catboost for training
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

# Train pool for Catboost with text_features and cat_features
train_pool = Pool(
    data=X_train, 
    label=y_train, 
    cat_features=['brandName'],
    text_features=['name']
)

# test pool for Catboost
test_pool = Pool(
    data=X_test, 
    label=y_test, 
    cat_features=['brandName'],
    text_features=['name']
)

# Initialize the CatBoostClassifier for GPU training.
model = CatBoostClassifier(
    iterations=1000,           
    task_type='GPU',           
    devices='0',               
    verbose=10,
    random_state=42,
    eval_metric='Accuracy',
    early_stopping_rounds=50,
    loss_function='MultiClass'
)

# Train the model using the training pool.
# Here, we're also providing an eval_set (optional) and enabling early stopping.
model.fit(train_pool, early_stopping_rounds=50)

# Once the model is trained, you can manually calculate predictions on the test set.
y_pred = model.predict(X_test)

# Example: print a few predictions.
accuracy_score(y_test, y_pred)

Learning rate set to 0.066901
0:	learn: 0.4491379	total: 175ms	remaining: 2m 54s
10:	learn: 0.5025862	total: 1.47s	remaining: 2m 11s
20:	learn: 0.5543103	total: 2.81s	remaining: 2m 10s
30:	learn: 0.5801724	total: 4.07s	remaining: 2m 7s
40:	learn: 0.6215517	total: 5.38s	remaining: 2m 5s
50:	learn: 0.6387931	total: 6.71s	remaining: 2m 4s
60:	learn: 0.6456897	total: 8.04s	remaining: 2m 3s
70:	learn: 0.6482759	total: 9.32s	remaining: 2m 2s
80:	learn: 0.6568966	total: 10.6s	remaining: 2m
90:	learn: 0.6603448	total: 12s	remaining: 2m
100:	learn: 0.6655172	total: 13.4s	remaining: 1m 59s
110:	learn: 0.6706897	total: 14.8s	remaining: 1m 58s
120:	learn: 0.6767241	total: 16.1s	remaining: 1m 57s
130:	learn: 0.6870690	total: 17.4s	remaining: 1m 55s
140:	learn: 0.6991379	total: 18.9s	remaining: 1m 55s
150:	learn: 0.7060345	total: 20.3s	remaining: 1m 54s
160:	learn: 0.7086207	total: 21.6s	remaining: 1m 52s
170:	learn: 0.7163793	total: 22.9s	remaining: 1m 51s
180:	learn: 0.7284483	total: 24.4s	remaini

0.738831615120275

In [5]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
print(classification_report(y_test, y_pred))

                                precision    recall  f1-score   support

                          Acne       0.00      0.00      0.00         1
        Anti-Aging Facial Care       0.00      0.00      0.00         2
               Anti-Aging Mask       0.00      0.00      0.00         1
              Anti-Aging Serum       0.75      0.60      0.67         5
                  Baby & Child       0.00      0.00      0.00         1
                     Bleaching       0.00      0.00      0.00         2
                   Body Butter       0.00      0.00      0.00         1
                   Body Lotion       0.50      0.20      0.29         5
           Body Sun Protection       0.00      0.00      0.00         0
               Cleansing Cream       0.00      0.00      0.00         3
                 Cleansing Gel       0.00      0.00      0.00         0
                   Color Rinse       0.00      0.00      0.00         1
                     Concealer       0.50      1.00      0.67  

/home/kamesh/interview-tests/qogita/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kamesh/interview-tests/qogita/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kamesh/interview-tests/qogita/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{met

In [6]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,model.predict(X_test))

0.738831615120275

In [13]:
X_test.iloc[0]

name         Wella Koleston Perfect Me+ Rich Naturals Hair ...
brandName                                                Wella
Name: 538, dtype: object

In [25]:

input_df = pd.DataFrame({
    "name": "Filorga Ncef-Revitalize Polyrevitalizing Firming Effect Cream 50ml",
    "brandName": "Filorga",
    
},index=[0])

In [26]:
input_df

,name,brandName
0,Filorga Ncef-Revitalize Polyrevitalizing Firmi...,Filorga


In [28]:
model.predict(input_df)[0][0]

'Face Cream'

In [ ]:
# model.save_model('catboost_model_without_image.cbm')